In [2]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [13]:
co=gto.Mole(atom='H 0 0 0; H 0 0 1.2', unit="Bohr",basis="3-21g")
hfco=scf.RHF(co)
hfco.scf()

dL=.01
hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
print(np.allclose(dV,hf1.get_hcore()-hfco.get_hcore()))

converged SCF energy = -1.11377437897079
converged SCF energy = -1.11394322077261
converged SCF energy = -1.11394322077261
True


Initialize <pyscf.gto.mole.Mole object at 0x7f06fbb054e0> in <pyscf.scf.hf.RHF object at 0x7f06fbb05940>


In [14]:
C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1()
dP=(P1-P2)/2
dC=(abs(C1)*C/abs(C)-C)

In [15]:
h2=hfco.get_veff()
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
print( "J-K/2 = H2 :",np.allclose(h2,J-K/2))

J-K/2 = H2 : True


In [16]:
U_app=C.T.dot(dV.dot(C))  # approximate value of U
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_app[i][j]/=(e[j]-e[i])
dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
#plt.matshow(dP_app)
#plt.matshow(dP)
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.013112700507594336
0.011386593282550352
0.0020933958012161456


In [17]:
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
for i in range(20):
    U_app=C.T.dot((dV+np.einsum('ijkl,kl->ij',g_h2,dP_app)).dot(C)) # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
    #plt.matshow(dP_app)
    #plt.matshow(dP)
print(np.linalg.norm(dP))
print(np.linalg.norm(dP_app))
print(np.linalg.norm(dP_app-dP))

0.013112700507594336
0.013113572371241264
1.5943139887907726e-06


In [20]:
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
for i in range(10):
    U_app=C.T.dot((dV+np.einsum('ijkl,kl->ij',g_h2,dP_app)).dot(C)) # approximate value of U
    for i in range(co.nao):
        for j in range(co.nao):
            if e[i]!=e[j]:
                U_app[i][j]/=(e[j]-e[i])
    dP_app=C.dot(U_app.dot(O)-O.dot(U_app)).dot(C.T)
    #plt.matshow(dP_app)
    #plt.matshow(dP)
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))
    print(np.linalg.norm(dP_app-dP))

0.013112700507594336
0.013113572371243063
1.5943139925051544e-06
0.013112700507594336
0.01311357237124354
1.5943139934895672e-06
0.013112700507594336
0.013113572371243665
1.5943139937539878e-06
0.013112700507594336
0.013113572371243698
1.5943139938199181e-06
0.013112700507594336
0.013113572371243708
1.5943139938415666e-06
0.013112700507594336
0.013113572371243708
1.5943139938454977e-06
0.013112700507594336
0.01311357237124371
1.594313993845482e-06
0.013112700507594336
0.013113572371243713
1.5943139938484096e-06
0.013112700507594336
0.013113572371243712
1.5943139938471955e-06
0.013112700507594336
0.01311357237124371
1.5943139938462532e-06
